# Generate daily prices using daily data service with or without corporate actions adjustments - Python

### Overview
Daily prices is designed to illustrate price movements for an instrument over a look back period.

This sample demonstrates how to directly request and plot daily prices.<br>

It enables to retrieve **on-demand** daily prices data by calling a dedicated service on a `daily data store`. 

### Inputs/outputs
Daily prices sample requires instrument's identifier as per input. It returns daily prices and volumes.
This sample shows the differences when adjusting the prices with corporate actions.

### Services used
This sample uses *gRPC requests* in order to retrieve daily bars from the dedicated hosted service. The queried endpoint in this script are:
* *DailyPricesService*: to directly retrieve daily price objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.type.shared.v1*
    * *systemathics.apis.services.daily.v1*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matplotlib* as per display package

***

# Run intraday bars sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas mplfinance plotly systemathics.apis

In [ ]:
import os
import csv
import grpc
import pandas as pd
from datetime import datetime
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.daily.v1.daily_prices_pb2 as daily_prices
import systemathics.apis.services.daily.v1.daily_prices_pb2_grpc as daily_prices_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily prices* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyPricesService* in order to retrieve daily data

In [ ]:
# set the instrument
ticker = 'AAPL'
exchange = 'XNGS'

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
# Non adjusted
request = daily_prices.DailyPricesRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    adjustment = False
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the daily prices service
        service = daily_prices_service.DailyPricesServiceStub(channel)
        
        # process the daily prices request
        response = service.DailyPrices(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# With adjustments
request_adjusted = daily_prices.DailyPricesRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    adjustment = True
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the daily prices service
        service = daily_prices_service.DailyPricesServiceStub(channel)
        
        # process the daily prices request
        response_adjusted = service.DailyPrices(
            request = request_adjusted, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
print("Total bars retrieved without adjustments: ", len(response.data))
print("Total bars retrieved with adjustments: ", len(response_adjusted.data))

### Step 4: Export data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
#pPrepare the data frame content
dates=[datetime(p.date.year, p.date.month, p.date.day) for p in response.data]
prices = [p.price for p in response.data]
volumes = [p.volume for p in response.data]
scores = [p.score for p in response.data]

prices_adjusted = [p.price for p in response_adjusted.data]
volumes_adjusted = [p.volume for p in response_adjusted.data]
scores_adjusted = [p.score for p in response_adjusted.data]

d = {'Date': dates, 'Price': prices,'Volume': volumes, 'Score': scores, 'Price_adjusted': prices_adjusted, 'Volume_adjusted':volumes_adjusted, 'Score_adjusted': scores_adjusted}

In [ ]:
# create pandas dataframe
df = pd.DataFrame(data=d)
df

In [ ]:
filename = 'Export/{0}-{1}-dailyprices-comparison.csv'.format(ticker, exchange)
df.to_csv(filename, index=False)

### 5 Plot prices in both cases
The following code snippets display daily bars' request results with the package of your choice.<br>*`plotly`* is used as per open source display package.

#### 5.1 Plot non adjusted daily prices

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Set date as index
#df = df.set_index('Date')
#df

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price'), secondary_y=True)
fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], name="Size"), secondary_y=False)

fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Daily prices and sizes for {0}-{1} without corporate actions adjustments'.format(ticker,exchange))
fig.layout.yaxis2.showgrid=False
fig.show()

#### 5.2 Plot daily prices adjusted with corporate actions

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df['Date'], y=df['Price_adjusted'], name='Price_adjusted'), secondary_y=True)
fig.add_trace(go.Bar(x=df['Date'], y=df['Volume_adjusted'], name="Size_adjusted"), secondary_y=False)

fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Daily prices and sizes for {0}-{1} with corporate actions adjustments'.format(ticker,exchange))
fig.layout.yaxis2.showgrid=False
fig.show()